In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
path = "/data/lux70/data/pfam/train.fasta"
from evo.dataset import FastaDataset
from plaid.datasets import MetadataParser

In [6]:
ds = FastaDataset(path)

100%|██████████| 9.03G/9.03G [00:33<00:00, 291MB/s]
100%|██████████| 9.03G/9.03G [00:51<00:00, 188MB/s]


In [7]:
headers = []
sequences = []

from tqdm.notebook import tqdm, trange

for i in trange(len(ds)):
    header, sequence = ds[i]
    headers.append(header)
    sequences.append(sequence)

  0%|          | 0/44190382 [00:00<?, ?it/s]

In [8]:
import pandas as pd

df = pd.DataFrame({
    "sequences": sequences,
    "headers": headers
})

In [12]:
parser = MetadataParser()

df['pfam_id'] = df.headers.map(lambda x: parser.header_to_pfam_id(x))
df['organism_id'] = df.headers.map(lambda x: parser.header_to_organism(x))
df['organism_idx'] = df.headers.map(lambda x: parser.header_to_organism_idx(x))
# df['GO_idx'] = df.headers.map(lambda x: parser.header_to_go_idx(x))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f1d06c8a430>>
Traceback (most recent call last):
  File "/homefs/home/lux70/micromamba/envs/omegafold/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [13]:
df.head()

,sequences,headers,pfam_id,organism_idx,organism_id
0,AFQYVQANPDEVCPANWTPGEKTMKPDPEGSKEYFSA,Q3AXS2_SYNS9/165-201 Q3AXS2.1 PF10417.13;1-cys...,PF10417,SYNS9,SYNS9
1,GLQYNRKTGEVVPANWVPGQSGITRDTKYIGR,K6DUV5_SCHAZ/104-135 K6DUV5.1 PF10417.13;1-cys...,PF10417,SCHAZ,SCHAZ
2,AAQYVRTHPGEVCPAKWKEDGETLKPSFDLVGK,A0A2C9XNY2_9ENTE/154-186 A0A2C9XNY2.1 PF10417....,PF10417,9ENTE,9ENTE
3,AFQYATAHPGEVCPANWKPGHKTIKADQDAKYDFFDA,W4HAV8_APHAT/228-264 W4HAV8.1 PF10417.13;1-cys...,PF10417,APHAT,APHAT
4,AAQYTAAHPNEVCPAKWKEGEKTLKPSLDLVGK,A0A4R5TJJ5_9GAMM/154-186 A0A4R5TJJ5.1 PF10417....,PF10417,9GAMM,9GAMM


In [14]:
go_df = parser.go_metadata
go_df.head()

,pfam_id,GO_id,GO_term,GO_level,count,GO_idx
0,PF00001,GO:0004930,G protein-coupled receptor activity,function,11,0
1,PF00002,GO:0004930,G protein-coupled receptor activity,function,11,0
2,PF00003,GO:0004930,G protein-coupled receptor activity,function,11,0
3,PF00004,GO:0016887,ATP hydrolysis activity,function,15,1
4,PF00005,GO:0016887,ATP hydrolysis activity,function,15,1


In [19]:
df_ = df.copy()

In [20]:
df = df.merge(go_df, on="pfam_id", how="left")

In [21]:
print(df.shape)
df.head()

(44190382, 10)


,sequences,headers,pfam_id,organism_idx,organism_id,GO_id,GO_term,GO_level,count,GO_idx
0,AFQYVQANPDEVCPANWTPGEKTMKPDPEGSKEYFSA,Q3AXS2_SYNS9/165-201 Q3AXS2.1 PF10417.13;1-cys...,PF10417,SYNS9,SYNS9,GO:0051920,peroxiredoxin activity,function,2.0,829.0
1,GLQYNRKTGEVVPANWVPGQSGITRDTKYIGR,K6DUV5_SCHAZ/104-135 K6DUV5.1 PF10417.13;1-cys...,PF10417,SCHAZ,SCHAZ,GO:0051920,peroxiredoxin activity,function,2.0,829.0
2,AAQYVRTHPGEVCPAKWKEDGETLKPSFDLVGK,A0A2C9XNY2_9ENTE/154-186 A0A2C9XNY2.1 PF10417....,PF10417,9ENTE,9ENTE,GO:0051920,peroxiredoxin activity,function,2.0,829.0
3,AFQYATAHPGEVCPANWKPGHKTIKADQDAKYDFFDA,W4HAV8_APHAT/228-264 W4HAV8.1 PF10417.13;1-cys...,PF10417,APHAT,APHAT,GO:0051920,peroxiredoxin activity,function,2.0,829.0
4,AAQYTAAHPNEVCPAKWKEGEKTLKPSLDLVGK,A0A4R5TJJ5_9GAMM/154-186 A0A4R5TJJ5.1 PF10417....,PF10417,9GAMM,9GAMM,GO:0051920,peroxiredoxin activity,function,2.0,829.0


In [23]:
df.GO_id.isna().sum() / len(df)

0.5336580254047136

In [24]:
# import pandas as pd

# df.to_parquet("/data/lux70/data/pfam/train.parquet")

In [25]:
path = "/data/lux70/plaid/artifacts/samples/5j007z42/100_200_300_v2/100/designability.csv"
desdf = pd.read_csv(path)

In [35]:
from pathlib import Path

def _ensure_parent_exists(path):
    path = Path(path)
    if not path.parent.exists():
        path.parent.mkdir(parents=True)

def copy(src, dst):
    _ensure_parent_exists(dst)
    shutil.copy2(src, dst)


def move(src, dst):
    _ensure_parent_exists(dst)
    shutil.move(src, dst)


def move_designable(df, delete_original=False):
    generated_pdb_paths = df.pdb_paths

    if delete_original:
        for i, p in enumerate(generated_pdb_paths):
            if df['designable'][i]:
                move(p, p.replace("generated/structures", "designable"))
            else:
                move(p, p.replace("generated/structures", "undesignable"))
    
    else:
        for i, p in enumerate(generated_pdb_paths):
            if df['designable'][i]:
                copy(p, p.replace("generated/structures", "designable"))
            else:
                copy(p, p.replace("generated/structures", "undesignable"))



In [36]:
import shutil

move_designable(desdf)